# Truss model

> Creation of one truss model for structural analysis with IfcOpenShell

In [ ]:
# | default_exp _truss_creation

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

The package ifcopenshell will be used to interact with IFC-SPF files. For more information about all the formats IFC has, head to this site:  
https://technical.buildingsmart.org/standards/ifc/ifc-formats/

For a tutorial about the ifcopenshell package, head to this site:  
https://blenderbim.org/docs-python/ifcopenshell-python/hello_world.html.

In [ ]:
# | export
from fastcore.basics import patch
import ifcopenshell
import pandas as pd

`TrussCreation` will contain all methods and attributes to create the IFC Entity's from a DataFrame which has the information nodes, members, member area, member e-module, support conditions and load that are relevant for a truss system.

In [ ]:
# | export
class TrussCreation:
    def __init__(
        self,
        *,
        model: ifcopenshell.file,
        representation_contexts,
        world_coordinate_system,
        relating_context,
        Bars,
        Nodes,
        Point_Loads,
        Load_Groups,
    ):
        self.ifc_model = model
        self.representation_contexts = representation_contexts
        self.world_coordinate_system = world_coordinate_system
        self.bars = Bars
        self.nodes = Nodes
        self.point_loads = Point_Loads
        self.load_groups = Load_Groups

        # IfcStructuralAnalysisModel
        self.STRUCTURAL_ANALYSIS_MODEL_PREDEFINED_TYPE = "NOTDEFINED"
        self.STRUCTURAL_ANALYSIS_MODEL_NAME = "Truss model"

        # IfcRelDeclares
        self.REL_DECLARES_RELATING_CONTEXT = relating_context

        self.STRUCTURAL_CURVE_MEMBER_PREDEFINED_TYPE = "PIN_JOINED_MEMBER"
        self.ifc_true = self.ifc_model.createIfcBoolean(True)
        self.ifc_flase = self.ifc_model.createIfcBoolean(False)
        self.JOINT = self.ifc_model.createIfcBoundaryNodeCondition(
            "Joint",
            self.ifc_true,  # TranslationalStiffnessX
            self.ifc_true,  # TranslationalStiffnessY
            self.ifc_true,  # TranslationalStiffnessZ
            self.ifc_flase,  # RotationalStiffnessX
            self.ifc_flase,  # RotationalStiffnessY
            self.ifc_flase,  # RotationalStiffnessZ
        )
        self.bars_and_nodes = []

In [ ]:
# | export
@patch
def create_structural_analysis_model(
    self: TrussCreation,
):
    self.orientation_of_2D_plane = self.world_coordinate_system
    self.shared_placement = self.ifc_model.createIfcLocalPlacement(
        None, self.orientation_of_2D_plane
    )

    # Structural analysis model
    self.ifc_structural_analysis_model = (
        self.ifc_model.createIfcStructuralAnalysisModel(
            ifcopenshell.guid.new(),
            None,
            self.STRUCTURAL_ANALYSIS_MODEL_NAME,
            None,
            None,
            self.STRUCTURAL_ANALYSIS_MODEL_PREDEFINED_TYPE,
            self.orientation_of_2D_plane,
            None,  # LoadedBy
            None,  # HasResults
            self.shared_placement,
        )
    )

    self.ifc_model.createIfcRelDeclares(
        ifcopenshell.guid.new(),
        None,
        None,  # Name
        None,
        self.REL_DECLARES_RELATING_CONTEXT,
        [self.ifc_structural_analysis_model],
    )

    self.ifc_object_placement = self.ifc_model.createIfcLocalPlacement(
        self.ifc_structural_analysis_model.SharedPlacement,
        self.ifc_structural_analysis_model.OrientationOf2DPlane,
    )

In [ ]:
# | export
@patch
def create_node(
    self: TrussCreation,
    ifc_object_placement,
    x: float,
    y: float,
    z: float,
    TranslationalStiffnessX: bool,
    TranslationalStiffnessY: bool,
    TranslationalStiffnessZ: bool,
    name: str = "Node",
):
    applied_condition = self.ifc_model.createIfcBoundaryNodeCondition(
        "Support",
        self.ifc_model.createIfcBoolean(TranslationalStiffnessX),
        self.ifc_model.createIfcBoolean(TranslationalStiffnessY),
        self.ifc_model.createIfcBoolean(TranslationalStiffnessZ),
        self.ifc_flase,  # RotationalStiffnessX
        self.ifc_flase,  # RotationalStiffnessY
        self.ifc_flase,  # RotationalStiffnessZ
    )

    vertex_geometry = self.ifc_model.createIfcCartesianPoint((x, y, z))
    vertex = self.ifc_model.createIfcVertexPoint(vertex_geometry)
    items = [vertex]
    representations = [
        self.ifc_model.createIfcTopologyRepresentation(
            self.representation_contexts,
            "Reference",
            "Vertex",
            items,
        )
    ]
    representation = self.ifc_model.createIfcProductDefinitionShape(
        None,
        None,
        representations,
    )
    node = self.ifc_model.createIfcStructuralPointConnection(
        ifcopenshell.guid.new(),
        None,
        name,
        None,
        None,
        ifc_object_placement,
        representation,
        applied_condition,
        None,
    )

    self.bars_and_nodes.append(node)

    return node, vertex

In [ ]:
# | export
@patch
def create_nodes(
    self: TrussCreation,
):
    self.ifc_nodes = pd.DataFrame(
        [
            (
                name,
                *(
                    self.create_node(
                        self.ifc_object_placement,
                        *(x, y, z),  # Coordinate
                        *(xx, yy, zz),  # Translational
                        f"Node {name}",
                    )
                ),
            )
            for x, y, z, xx, yy, zz, name in zip(
                self.nodes["Coordinate_X"],
                self.nodes["Coordinate_Y"],
                self.nodes["Coordinate_Z"],
                self.nodes["Translational_X"],
                self.nodes["Translational_Y"],
                self.nodes["Translational_Z"],
                self.nodes["Node"],
            )
        ],
        columns=["Node", "Ifc_Node", "Ifc_Vertex"],
    )

In [ ]:
# | export
@patch
def create_bar(
    self: TrussCreation,
    ifc_object_placement,
    start_node,
    start_vertex,
    end_node,
    end_vertex,
    name: str = "Truss member",
):
    items = [
        self.ifc_model.createIfcEdge(
            EdgeStart=start_vertex, EdgeEnd=end_vertex
        )
    ]

    # EdgeStart to EdgeEnd represents the local x-axis

    representations = [
        self.ifc_model.createIfcTopologyRepresentation(
            self.representation_contexts,
            "Reference",
            "Edge",
            items,
        )
    ]
    representation = self.ifc_model.createIfcProductDefinitionShape(
        None,
        None,
        representations,
    )

    bar = self.ifc_model.createIfcStructuralCurveMember(
        ifcopenshell.guid.new(),
        None,
        name,
        None,
        None,
        ifc_object_placement,
        representation,
        self.STRUCTURAL_CURVE_MEMBER_PREDEFINED_TYPE,
        self.ifc_model.createIfcDirection(
            (1.0, 0.0, 0.0)
        ),  # Local z axis from member -> needs to be calculatet from nodes
    )

    for node in [start_node, end_node]:
        self.ifc_model.createIfcRelConnectsStructuralMember(
            ifcopenshell.guid.new(),
            None,
            "Joint",
            None,
            bar,
            node,
            self.JOINT,
            None,
            None,
            None,
        )
    self.bars_and_nodes.append(bar)

    return bar

In [ ]:
# | export
@patch
def create_bars(
    self: TrussCreation,
):
    ifc_start = self.ifc_nodes.rename(
        columns={
            "Node": "Start_node",
            "Ifc_Node": "Ifc_start_node",
            "Ifc_Vertex": "Ifc_start_vertex",
        }
    )
    ifc_bars_start = pd.merge(self.bars, ifc_start, on="Start_node")
    ifc_end = self.ifc_nodes.rename(
        columns={
            "Node": "End_node",
            "Ifc_Node": "Ifc_end_node",
            "Ifc_Vertex": "Ifc_end_vertex",
        }
    )
    ifc_bars_start_and_end = pd.merge(
        ifc_bars_start, ifc_end, on="End_node"
    )

    ifc_bars = [
        (
            name,
            self.create_bar(
                self.ifc_object_placement,
                *(start_node, start_vertex, end_node, end_vertex),
            ),
        )
        for start_node, start_vertex, end_node, end_vertex, name in zip(
            ifc_bars_start_and_end["Ifc_start_node"],
            ifc_bars_start_and_end["Ifc_start_vertex"],
            ifc_bars_start_and_end["Ifc_end_node"],
            ifc_bars_start_and_end["Ifc_end_vertex"],
            ifc_bars_start_and_end["Bar"],
        )
    ]
    self.ifc_bars = pd.DataFrame(ifc_bars, columns=["Bar", "Ifc_Bar"])

In [ ]:
# | export
@patch
def assignment_of_nodes_and_bars_to_the_analysis_model(
    self: TrussCreation,
):
    self.ifc_model.createIfcRelAssignsToGroup(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        self.bars_and_nodes,
        None,
        self.ifc_structural_analysis_model,
    )

In [ ]:
# | export
@patch
def create_point_load(
    self: TrussCreation,
    name,
    ifc_node,
    ForceX,
    ForceY,
    ForceZ,
):
    STRUCTURAL_POINT_ACTION_GLOBAL_OR_LOCAL = "GLOBAL_COORDS"

    single_force = self.ifc_model.createIfcStructuralLoadSingleForce(
        name,
        ForceX,
        ForceY,
        ForceZ,
        None,  # MomentX
        None,  # MomentY
        None,  # MomentZ
    )
    point_load = self.ifc_model.createIfcStructuralPointAction(
        ifcopenshell.guid.new(),
        None,
        name,
        None,
        None,
        None,
        None,
        single_force,
        STRUCTURAL_POINT_ACTION_GLOBAL_OR_LOCAL,
        None,
    )
    self.ifc_model.createIfcRelConnectsStructuralActivity(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        ifc_node,
        point_load,
    )

    return point_load

In [ ]:
# | export
@patch
def create_point_loads(
    self: TrussCreation,
):
    self.point_loads_expanded = pd.merge(
        self.point_loads, self.ifc_nodes, on="Node"
    )
    ifc_point_loads = [
        (
            name,
            self.create_point_load(
                f"Point Load {name}",
                node,
                ForceX,
                ForceY,
                ForceZ,
            ),
        )
        for name, node, ForceX, ForceY, ForceZ in zip(
            self.point_loads_expanded["Point_Load"],
            self.point_loads_expanded["Ifc_Node"],
            self.point_loads_expanded["Force_X"],
            self.point_loads_expanded["Force_Y"],
            self.point_loads_expanded["Force_Z"],
        )
    ]
    self.ifc_point_loads = pd.DataFrame(
        ifc_point_loads,
        columns=["Point_Load", "Ifc_Point_Load"],
    )

In [ ]:
# | export
@patch
def create_load_group(self: TrussCreation, name, ifc_point_loads):
    STRUCTURAL_LOAD_GROUP_PREDEFINED_TYPE = "LOAD_GROUP"
    # Normally needed if 'PredefinedType' specifies a LOAD_CASE
    STRUCTURAL_LOAD_GROUP_ACTION_TYPE = "NOTDEFINED"
    STRUCTURAL_LOAD_GROUP_ACTION_SOURCE = "NOTDEFINED"
    load_group = self.ifc_model.createIfcStructuralLoadGroup(
        ifcopenshell.guid.new(),
        None,
        name,
        None,
        None,
        STRUCTURAL_LOAD_GROUP_PREDEFINED_TYPE,
        STRUCTURAL_LOAD_GROUP_ACTION_TYPE,
        STRUCTURAL_LOAD_GROUP_ACTION_SOURCE,
        None,
        None,
    )

    self.ifc_model.createIfcRelAssignsToGroup(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        ifc_point_loads,
        None,
        load_group,
    )

    return load_group

In [ ]:
# | export
@patch
def create_load_groups(
    self: TrussCreation,
):
    self.load_groups_expanded = pd.merge(
        self.load_groups, self.ifc_point_loads, on="Point_Load"
    )

    ifc_loads_groups = [
        (
            load_group,
            self.create_load_group(
                f"Load Group {load_group}",
                sub_df["Ifc_Point_Load"].tolist(),
            ),
        )
        for load_group, sub_df in self.load_groups_expanded.groupby(
            "Load_Group"
        )
    ]

    self.ifc_loads_groups = pd.DataFrame(
        ifc_loads_groups,
        columns=["Load_Group", "Ifc_Load_Group"],
    )

    self.ifc_structural_analysis_model.LoadedBy = self.ifc_loads_groups[
        "Ifc_Load_Group"
    ].tolist()

In [ ]:
# | export
@patch
def create_modulus_of_elasticity(
    self: TrussCreation,
    modulus_of_elasticity,
):
    MATERIAL_NAME = "Modulus of elasticity"
    ifc_material = self.ifc_model.createIfcMaterial(
        MATERIAL_NAME,  # Not optional
        None,  # optional
        None,  # optional
    )

    ifc_modulus_of_elasticity = self.ifc_model.createIfcPropertySingleValue(
        "YoungModulus",
        None,
        self.ifc_model.createIfcModulusOfElasticityMeasure(
            modulus_of_elasticity
        ),
        None,
    )

    self.ifc_model.createIfcMaterialProperties(
        "Pset_MaterialMechanical",
        None,
        [ifc_modulus_of_elasticity],
        ifc_material,
    )

    return ifc_material

In [ ]:
# | export
@patch
def create_modulus_of_elasticitys(
    self: TrussCreation,
):
    ifc_materials = [
        (
            modulus_of_elasticity,
            self.create_modulus_of_elasticity(
                modulus_of_elasticity,
            ),
        )
        for modulus_of_elasticity in self.bars[
            "Modulus_of_elasticity"
        ].unique()
    ]

    self.ifc_materials = pd.DataFrame(
        ifc_materials, columns=["Modulus_of_elasticity", "ifc_material"]
    )

In [ ]:
# | export
@patch
def create_surface_area(
    self: TrussCreation,
    surface_area,
):
    PROFILE_DEF_PROFILE_TYPE = "AREA"
    ifc_profile = self.ifc_model.createIfcProfileDef(
        PROFILE_DEF_PROFILE_TYPE,
        None,
    )
    ifc_surface_area = self.ifc_model.createIfcPropertySingleValue(
        "CrossSectionArea",
        None,
        self.ifc_model.createIfcAreaMeasure(surface_area),
        None,
    )

    self.ifc_model.createIfcProfileProperties(
        "Pset_ProfileMechanical", None, [ifc_surface_area], ifc_profile
    )

    return ifc_profile

In [ ]:
# | export
@patch
def create_surface_areas(
    self: TrussCreation,
):
    ifc_profiles = [
        (
            surface_area,
            self.create_surface_area(
                surface_area,
            ),
        )
        for surface_area in self.bars["Cross-sectional_area"].unique()
    ]

    self.ifc_profiles = pd.DataFrame(
        ifc_profiles, columns=["Cross-sectional_area", "ifc_profile"]
    )

In [ ]:
# | export
@patch
def create_material_profile(
    self: TrussCreation, ifc_profile, ifc_material, ifc_bars
):
    ifc_material_profile = self.ifc_model.createIfcMaterialProfile(
        None,
        None,
        ifc_material,
        ifc_profile,
        None,
        None,
    )

    ifc_material_profile_set = self.ifc_model.createIfcMaterialProfileSet(
        None, None, [ifc_material_profile], None
    )
    self.ifc_model.createIfcRelAssociatesMaterial(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        ifc_bars,
        ifc_material_profile_set,
    )
    return ifc_material_profile

In [ ]:
# | export
@patch
def create_material_profiles(
    self: TrussCreation,
):
    bars_extended = pd.merge(self.bars, self.ifc_bars, on="Bar")
    bars_extended = pd.merge(
        bars_extended, self.ifc_materials, on="Modulus_of_elasticity"
    )
    bars_extended = pd.merge(
        bars_extended, self.ifc_profiles, on="Cross-sectional_area"
    )

    ifc_material_profiles = [
        (
            *group,
            self.create_material_profile(
                sub_df["ifc_profile"].unique()[0],
                sub_df["ifc_material"].unique()[0],
                sub_df["Ifc_Bar"].tolist(),
            ),
        )
        for group, sub_df in bars_extended.groupby(
            by=["Cross-sectional_area", "Modulus_of_elasticity"]
        )
    ]

    self.ifc_material_profiles = pd.DataFrame(
        ifc_material_profiles,
        columns=[
            "Cross-sectional_area",
            "Modulus_of_elasticity",
            "ifc_material_profile",
        ],
    )

In [ ]:
Nodes_data = {
    "Node": pd.Series([1, 2, 3, 4], dtype=int),
    "Coordinate_X": pd.Series([0, 0, -4e3, -4e3], dtype=float),
    "Coordinate_Y": pd.Series([0, 0, 0, 0], dtype=float),
    "Coordinate_Z": pd.Series([0, 3e3, 3e3, 6e3], dtype=float),
    "Translational_X": pd.Series([1, 0, 1, 1], dtype=bool),
    "Translational_Y": pd.Series([1, 0, 1, 1], dtype=bool),
    "Translational_Z": pd.Series([1, 0, 1, 1], dtype=bool),
}

Nodes = pd.DataFrame(Nodes_data)
Nodes.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z
1,0.000000,0.000000,0.000000,True,True,True
2,0.000000,0.000000,3000.000000,False,False,False
3,-4000.000000,0.000000,3000.000000,True,True,True
4,-4000.000000,0.000000,6000.000000,True,True,True


In [ ]:
Bars_data = {
    "Bar": pd.Series([1, 2, 3], dtype=int),
    "Start_node": pd.Series([2, 2, 2], dtype=int),
    "End_node": pd.Series([1, 3, 4], dtype=int),
    "Cross-sectional_area": pd.Series([1e3, 2e3, 1e3], dtype=float),
    "Modulus_of_elasticity": pd.Series([1e3, 2e3, 2e3], dtype=float),
}
Bars = pd.DataFrame(Bars_data)
Bars.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity
1,2,1,1000.000000,1000.000000
2,2,3,2000.000000,2000.000000
3,2,4,1000.000000,2000.000000


In [ ]:
Point_Loads_data = {
    "Point_Load": pd.Series(
        [
            1,
        ],
        dtype=int,
    ),
    "Node": pd.Series(
        [
            2,
        ],
        dtype=int,
    ),
    "Force_X": pd.Series(
        [
            100e3,
        ],
        dtype=float,
    ),
    "Force_Y": pd.Series(
        [
            0,
        ],
        dtype=float,
    ),
    "Force_Z": pd.Series(
        [
            -100e3,
        ],
        dtype=float,
    ),
}
Point_Loads = pd.DataFrame(Point_Loads_data)
Point_Loads.style.hide(axis="index")

Point_Load,Node,Force_X,Force_Y,Force_Z
1,2,100000.000000,0.000000,-100000.000000


In [ ]:
Load_Groups_data = {
    "Load_Group": pd.Series(
        [
            1,
        ],
        dtype=int,
    ),
    "Point_Load": pd.Series(
        [
            1,
        ],
        dtype=int,
    ),
}
Load_Groups = pd.DataFrame(Load_Groups_data)
Load_Groups.style.hide(axis="index")

Load_Group,Point_Load
1,1


In [ ]:
import ifctruss._structural_analysis_model

In [ ]:
truss = (
    ifctruss._structural_analysis_model.StructuralAnalysisModelCreation()
)
truss.create_basic_structure()

In [ ]:
truss_model = TrussCreation(
    model=truss.ifc_model,
    representation_contexts=truss.representation_contexts,
    world_coordinate_system=truss.world_coordinate_system,
    relating_context=truss.ifc_project,
    Bars=Bars,
    Nodes=Nodes,
    Point_Loads=Point_Loads,
    Load_Groups=Load_Groups,
)

In [ ]:
truss_model.create_structural_analysis_model()
truss_model.create_nodes()
truss_model.create_bars()
truss_model.assignment_of_nodes_and_bars_to_the_analysis_model()
truss_model.create_point_loads()
truss_model.create_load_groups()
truss_model.create_modulus_of_elasticitys()
truss_model.create_surface_areas()
truss_model.create_material_profiles()

In [ ]:
# | hide
nbdev.nbdev_export()